In [4]:
import re
import nltk
from textblob import TextBlob
from collections import Counter
import spacy
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize.treebank import TreebankWordDetokenizer as Detok
from textblob.wordnet import VERB
from nltk.tokenize import RegexpTokenizer

# 
# BW: This script can be used to extract know concepts (found in an ontology) from a transcript
#

#Open ontology-file 
ontologyFile = open('ontology/ontology.txt', 'r')
ontology = ontologyFile.read()
ontologyWords = ontology.split()

detokenizer= Detok()
speaker_words = {}
speaker_pattern = re.compile(r'^(\w+?):(.*)$')
#Get stopwords and extend this corpus
stopwords = nltk.corpus.stopwords.words('english')
stopwords.extend(['Yeah','yeah', 'Um','Well','Right', 'I', 'Yes','Uh','uh','Okay','okay', 'Sorry','sorry','Yep','yep','Uhh','uhh','Gonna', 'gonna','Gon na','gon na','Mm','mm','Wanna','Wan na','wanna','wan na','Ok','ok','OK','Oh','oh','Mhm','mhm','Sure','sure','Mhmm','mhmm','Jeez','jeez','Hi','hi','wow','Wow','True','true'])

#Go through transcript and split sentences based on the different speakers
with open("transcript-splitted/splitted-wd_d1_r2.txt", "r") as f:
        lines = f.readlines()
        current_speaker = None
        for line in lines:
                line = line.strip()
                match = speaker_pattern.match(line)
                if match is not None:
                        current_speaker = match.group(1)
                        line = match.group(2).strip()
                        if current_speaker not in speaker_words.keys():
                                speaker_words[current_speaker] = []
                if current_speaker:
                        words = [word.strip() for word in line.split(' ') if len(word.strip()) > 0]
                        speaker_words[current_speaker].extend(words)

dictget = lambda d, *k: [d[i] for i in k]
spk_0 = dictget(speaker_words, 'spk_0')

#Go through text of specific speaker, and count words if the word is found in the ontology words
text_0 = detokenizer.detokenize(spk_0)
count_0 = {w: 0 for w in ontologyWords}
for index, word in enumerate(nltk.word_tokenize(text_0)):
    if word in count_0:
        for w in word.split():
            count_0[w] +=1
#Create dataframe based on counted words
df_0 = pd.DataFrame.from_dict(count_0, orient='index').reset_index()
df_0.columns = ['Ontology Word', 'Frequency']
df_0.insert(2, "Speaker", "spk_0", True) 
df_0 = df_0.sort_values(by=['Frequency'], ascending=False)

#Combine different tables 
frames = [df_0]
result = pd.concat(frames)
result_sorted = result.sort_values(by=['Frequency'], ascending=False)
result_sorted.to_excel("output/known-words.xlsx", index=False)